In [ ]:
import tf_encrypted as tfe
import tensorflow as tf

config = tfe.RemoteConfig.load("/tmp/tfe.config")

tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

input_shape = (1, 224, 224, 3)
output_shape = (1, 10)

In [ ]:
client = tfe.serving.QueueClient(
    input_shape=input_shape,
    output_shape=output_shape)

In [ ]:
sess = tfe.Session(config=config)

In [ ]:
fpath = 'subject06_centerlight-0000.jpg'

In [ ]:
'''
import numpy as np
from keras import backend as K


def preprocess_input(x, data_format=None, version=1):
    x_temp = np.copy(x)
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if version == 1:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

    elif version == 2:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
    else:
        raise NotImplementedError

    return x_temp
'''

In [ ]:
_# face verification with the VGGFace2 model
#from matplotlib import pyplot
from PIL import Image
from numpy import asarray
#from scipy.spatial.distance import cosine
#from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
import tensorflow as tf
#from keras_vggface.utils import preprocess_input
import tf_encrypted as tfe 
import numpy as np
import time

#lamb1 = np.zeros((1, 224, 224, 3))
#data_np = np.asarray(lamb1, np.float32)
#data_tf = tf.convert_to_tensor(data_np, np.float32)


#tfe_model = tfe.keras.models.clone_model(model)

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224, 3)):
    # load image from file
    #pixels = pyplot.imread(filename)
    # create the detector, using default weights
    #detector = MTCNN()
    # detect faces in the image
    #results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    #x1, y1, width, height = results[0]['box']
    #x2, y2 = x1 + width, y1 + height
    # extract the face
    #face = pixels
    # resize pixels to the model size
    pixels = Image.open(filename).convert('RGB')
    image = np.asarray(pixels)
    image = np.resize(image, required_size)
    print(image.shape)
    #face_array = asarray(image)
    return image

# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    print(samples.shape)
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    print(samples)
    #print(samples.shape)
    
    starttime = time.time()

    res = client.run(
          sess,
          samples)

    print(res)

    predicted_label = np.argmax(res)
    print(predicted_label)
    
    print("Private prediction time is - ")
    print(time.time() - starttime)

    #samples = np.expand_dims(samples, -1)
    #print(samples.shape)
    #m = model
    #model = VGGFace(model='vgg16', include_top=False, \
    #                input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    #yhat = m.predict(samples)
    return

embeddings = get_embeddings([fpath])